<a href="https://colab.research.google.com/github/amiller411/uniwork/blob/big_data_cw3/Copy_of_COM739_17952_CW3_run_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initial Setup**

1. First, you will setup your CoLab environment. Run the cell below.

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download both anime.csv and rating.csv, and store it in your google drive. It is advisable to create a separate project folder, where you can store this dataset and also your code.

The script will give you the id of the two files in your drive.

In [3]:
file_list = drive.ListFile({'q': "'1Oi8cMnAfJVZH9-FyXGxwOrGGCIkkB7uy' in parents"}).GetList()
for f in file_list:
  print('title: %s, id: %s' % (f['title'], f['id']))

title: rating.csv, id: 1f76dQZxRB1fNaReBv_DnUDVkIXNm7mw9
title: anime.csv, id: 1TppJoj4QVJlc_HML20xmH847Brrw0Zfc


If you executed the cells below, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
# Change the id, if it differs from the one below.
id='1TppJoj4QVJlc_HML20xmH847Brrw0Zfc'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('anime.csv')

id='1f76dQZxRB1fNaReBv_DnUDVkIXNm7mw9'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('rating.csv')

Here is a list of packages that might be useful to you. 

**Student Activity: Add the packages you need to carry out your analysis here** 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Student Activity: Add your packages here.
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
import numpy as np
# from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
# from pyspark.sql.functions import col, size
# from pyspark.sql.functions import isnan, when, count, col

**This step initializes the Spark context.**

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a local runtime).

In [7]:
spark

## **From this point onwards, you are supposed to do the coding yourself. Follow the steps as mentioned below in its appropriate place.**

**1. Student Activity: Read the datasets here. You must write the script for the first question and explore both the files here.**

Q1. Identify and describe the number of columns in the two dataset files.

In [7]:
df_anime = spark.read.csv("anime.csv", header=True, inferSchema = True)
df_rat = spark.read.csv("rating.csv", header=True, inferSchema = True)

In [9]:
df_anime.printSchema()
df_rat.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



The Anime dataset contains 7 columns (anime_id, name, genre, type, episodes, rating and members) with anime_id and members being the only integer types, rating the only double type and the remaining types all strings.

**2. Student Activity: Preprocess the datasets here. You must write the script for the second question here. Make sure to check if the script is running is correctly or not**

Q2. Merge/Join/Combine the two datasets and identify the key common column that would you have performed? 

Key column = rating, full join

In [11]:
df_anime.show()

+--------+--------------------+--------------------+-----+--------+------+-------+
|anime_id|                name|               genre| type|episodes|rating|members|
+--------+--------------------+--------------------+-----+--------+------+-------+
|   32281|      Kimi no Na wa.|Drama, Romance, S...|Movie|       1|  9.37| 200630|
|    5114|Fullmetal Alchemi...|Action, Adventure...|   TV|      64|  9.26| 793665|
|   28977|            Gintama°|Action, Comedy, H...|   TV|      51|  9.25| 114262|
|    9253|         Steins;Gate|    Sci-Fi, Thriller|   TV|      24|  9.17| 673572|
|    9969|       Gintama&#039;|Action, Comedy, H...|   TV|      51|  9.16| 151266|
|   32935|Haikyuu!!: Karasu...|Comedy, Drama, Sc...|   TV|      10|  9.15|  93351|
|   11061|Hunter x Hunter (...|Action, Adventure...|   TV|     148|  9.13| 425855|
|     820|Ginga Eiyuu Densetsu|Drama, Military, ...|  OVA|     110|  9.11|  80679|
|   15335|Gintama Movie: Ka...|Action, Comedy, H...|Movie|       1|   9.1|  72534|
|   

In [37]:
df_anime.describe().show()

+-------+------------------+-----------------+------+-----+------------------+-----------------+-----------------+
|summary|          anime_id|             name| genre| type|          episodes|           rating|          members|
+-------+------------------+-----------------+------+-----+------------------+-----------------+-----------------+
|  count|             12294|            12294| 12232|12269|             12294|            12064|            12294|
|   mean|14058.221652838783|         166778.5|  null| null|12.382549774134182|6.473901690981445|18071.33886448674|
| stddev|11455.294700988177|330891.6746051493|  null| null| 46.86535196440979|1.026746306898068|54820.67692490701|
|    min|                 1|    &quot;0&quot;|Action|Movie|                 1|             1.67|                5|
|    max|             34527|                ◯|  Yaoi|   TV|           Unknown|             10.0|          1013917|
+-------+------------------+-----------------+------+-----+------------------+--

In [36]:
df_rat.describe().show()

+-------+------------------+-----------------+------------------+
|summary|           user_id|         anime_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|           7813737|          7813737|           7813737|
|   mean|36727.956744640884|8909.072104295294| 6.144029546937656|
| stddev|20997.946118973723| 8883.94963588107|3.7278004201098067|
|    min|                 1|                1|                -1|
|    max|             73516|            34519|                10|
+-------+------------------+-----------------+------------------+



In [9]:
df_rat.select([count(when(isnan('rating'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(rating) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [10]:
df_anime.select([count(when(isnan('rating'),True))]).show()

+--------------------------------------------+
|count(CASE WHEN isnan(rating) THEN true END)|
+--------------------------------------------+
|                                           0|
+--------------------------------------------+



In [8]:
# dataframe.filter(dataframe.ID!='4').show()
df_rating_dropped = df_rat.filter(df_rat.rating!='-1')
# filter null values in rating
# df_rating_dropped = df_rating_dropped.na.drop()

In [12]:
df_rating_dropped.count()

6337241

In [9]:
df_rating_dropped = df_rating_dropped.selectExpr("user_id as user_id","anime_id as anime_id_rating", "rating as rating_rating")

In [13]:
df_anime.count()

12294

In [14]:
df_rating_dropped.count()

6337241

In [15]:
6337241+12294

6349535

In [10]:
# i need a full outer join on anime_id with anime_id and ratings renamed so no ambigious errors
# empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"fullouter") 
outer_join_df = df_anime.join(df_rating_dropped, df_anime.anime_id == df_rating_dropped.anime_id_rating, "fullouter")

In [12]:
outer_join_df.count()

6339609

In [13]:
outer_join_df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- anime_id_rating: integer (nullable = true)
 |-- rating_rating: integer (nullable = true)



In [25]:
outer_join_df.count()

6339609

In [11]:
#filter out null values (2)
outer_join_df_filtered = outer_join_df.filter(outer_join_df.anime_id. isNotNull())
# df.filter(df.col_X. isNotNull()) 

In [27]:
outer_join_df_filtered.count()

6339607

In [14]:
outer_join_df.show()

+--------+------------+--------------------+----+--------+------+-------+-------+---------------+-------------+
|anime_id|        name|               genre|type|episodes|rating|members|user_id|anime_id_rating|rating_rating|
+--------+------------+--------------------+----+--------+------+-------+-------+---------------+-------------+
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     19|              1|           10|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     21|              1|            9|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     23|              1|            9|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     32|              1|           10|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     34|              1|            7|
|       1|Cowboy Bebop|Action, Adventure...|  TV|      26|  8.82| 486824|     43|              1|       

In [ ]:
joined_rating.show()

+--------+--------------------+--------------------+-------+--------+------+-------+-------+--------+------+
|anime_id|                name|               genre|   type|episodes|rating|members|user_id|anime_id|rating|
+--------+--------------------+--------------------+-------+--------+------+-------+-------+--------+------+
|   24259|Mechano: Scientif...|Comedy, Dementia,...|    OVA|       3|  2.55|    120|   null|    null|  null|
|   33860|Saru Kani Gassen ...|          Historical|    ONA|       1|  2.84|     38|   null|    null|  null|
|   23677|Manga Edo Erobanashi|Comedy, Hentai, H...|    OVA|       5|  2.86|    111|   null|    null|  null|
|   31268|       Miira no Yume|            Dementia|  Movie|       1|  2.97|     98|   null|    null|  null|
|   29972|    Hakuchuu Meikyuu|            Dementia|    ONA|       1|  3.02|     99|   null|    null|  null|
|   16786|Bulsajo Robot Pho...|Action, Mecha, Sh...|  Movie|       1|  3.11|    133|   null|    null|  null|
|   32834|Gra-P &am

**3. Student Activity: Now do some exploratory analysis. You must write the script for the third and fourth question here. Make sure to check if the script is running is correctly or not**

Q3. Find the top 10 anime based on user rating. Use tabular/graphical presentation to provide evidence of your analysis.

Q4. Find the top 10 genre based on user rating. Use tabular/graphical presentation to provide evidence of your analysis.

I will display the top 10 anime with all ratings from both csvs then with only using ratings from the ratings.csv and and only using ratings from anime.csv

In [12]:
# cast types of two ratings and add the contents of the columns together
outer_join_df = outer_join_df_filtered.withColumn("rating_rating", outer_join_df_filtered.rating_rating.cast('double'))

In [18]:
outer_join_df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- anime_id_rating: integer (nullable = true)
 |-- rating_rating: double (nullable = true)



In [16]:
# create dataframe with both rating columns, creates new columns for only columns in col_list
col_list = ["rating", "rating_rating"]
both_rating_cols_2 = outer_join_df_filtered.groupBy("anime_id").agg( *[F.collect_list(column).alias(column) for column in col_list if column !='anime_id' ]) 

In [66]:
both_rating_cols_2.show()

+--------+--------------------+--------------------+
|anime_id|              rating|       rating_rating|
+--------+--------------------+--------------------+
|       1|[8.82, 8.82, 8.82...|[10.0, 9.0, 9.0, ...|
|       5|[8.4, 8.4, 8.4, 8...|[9.0, 9.0, 10.0, ...|
|       6|[8.32, 8.32, 8.32...|[8.0, 7.0, 10.0, ...|
|      15|[8.08, 8.08, 8.08...|[6.0, 7.0, 9.0, 7...|
|      16|[8.18, 8.18, 8.18...|[9.0, 9.0, 8.0, 9...|
|      17|[7.74, 7.74, 7.74...|[6.0, 10.0, 9.0, ...|
|      19|[8.72, 8.72, 8.72...|[10.0, 7.0, 10.0,...|
|      20|[7.81, 7.81, 7.81...|[8.0, 6.0, 8.0, 9...|
|      22|[8.04, 8.04, 8.04...|[5.0, 7.0, 9.0, 7...|
|      26|[7.78, 7.78, 7.78...|[7.0, 9.0, 5.0, 9...|
|      27|[7.44, 7.44, 7.44...|[9.0, 8.0, 8.0, 6...|
|      28|[8.05, 8.05, 8.05...|[10.0, 9.0, 7.0, ...|
|      31|[7.51, 7.51, 7.51...|[9.0, 9.0, 6.0, 9...|
|      43|[8.34, 8.34, 8.34...|[7.0, 8.0, 8.0, 1...|
|      44|[8.83, 8.83, 8.83...|[7.0, 10.0, 10.0,...|
|      47|[8.15, 8.15, 8.15...|[8.0, 8.0, 7.0,

In [64]:
both_rating_cols.show()

+--------+--------------------+--------------------+
|anime_id|              rating|       rating_rating|
+--------+--------------------+--------------------+
|       1|8.828.828.828.828...|10.09.09.010.07.0...|
|       5|8.48.48.48.48.48....|9.09.010.09.09.09...|
|       6|8.328.328.328.328...|8.07.010.09.09.09...|
|      15|8.088.088.088.088...|6.07.09.07.09.07....|
|      16|8.188.188.188.188...|9.09.08.09.09.08....|
|      17|7.747.747.747.747...|6.010.09.09.010.0...|
|      19|8.728.728.728.728...|10.07.010.010.08....|
|      20|7.817.817.817.817...|8.06.08.09.09.06....|
|      22|8.048.048.048.048...|5.07.09.07.07.08....|
|      26|7.787.787.787.787...|7.09.05.09.07.07....|
|      27|7.447.447.447.447...|9.08.08.06.07.05....|
|      28|8.058.058.058.058...|10.09.07.08.08.08...|
|      31|7.517.517.517.517...|9.09.06.09.010.08...|
|      43|8.348.348.348.348...|7.08.08.010.09.08...|
|      44|8.838.838.838.838...|7.010.010.09.07.0...|
|      47|8.158.158.158.158...|8.08.07.07.09.0

now we have both ratings columns, concatenate the columns so we can then average them

In [19]:
all_ratings = both_rating_cols_2.withColumn("all_ratings", concat(col("rating"), col("rating_rating")))

In [85]:
all_ratings.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- rating: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- rating_rating: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- all_ratings: array (nullable = false)
 |    |-- element: double (containsNull = false)



In [13]:
# create function to calculate mean of an array (each row)
array_mean = udf(lambda x: float(np.mean(x)), FloatType())

In [20]:
# create average rating column applying array_mean function to all_ratings column
all_ratings_avg = all_ratings.withColumn("avg_rating", array_mean("all_ratings"))

In [43]:
all_ratings_avg.show()

+--------+--------------------+--------------------+--------------------+----------+
|anime_id|              rating|       rating_rating|         all_ratings|avg_rating|
+--------+--------------------+--------------------+--------------------+----------+
|       1|[8.82, 8.82, 8.82...|[10, 9, 9, 10, 7,...|[8.82, 8.82, 8.82...|  8.844716|
|       5|[8.4, 8.4, 8.4, 8...|[9, 9, 10, 9, 9, ...|[8.4, 8.4, 8.4, 8...|  8.419862|
|       6|[8.32, 8.32, 8.32...|[8, 7, 10, 9, 9, ...|[8.32, 8.32, 8.32...|  8.369697|
|      15|[8.08, 8.08, 8.08...|[6, 7, 9, 7, 9, 7...|[8.08, 8.08, 8.08...|   8.18985|
|      16|[8.18, 8.18, 8.18...|[9, 9, 8, 9, 9, 8...|[8.18, 8.18, 8.18...|  8.272468|
|      17|[7.74, 7.74, 7.74...|[6, 10, 9, 9, 10,...|[7.74, 7.74, 7.74...| 7.8917804|
|      19|[8.72, 8.72, 8.72...|[10, 7, 10, 10, 8...|[8.72, 8.72, 8.72...|  8.820652|
|      20|[7.81, 7.81, 7.81...|[8, 6, 8, 9, 9, 6...|[7.81, 7.81, 7.81...|  7.851944|
|      22|[8.04, 8.04, 8.04...|[5, 7, 9, 7, 7, 8...|[8.04, 8.04, 

In [21]:
# create a size column indicating how many raitings are in eacha array (row) so we can filter any anime with low numbers of ratings
all_ratings_avg = all_ratings_avg.withColumn("size", F.size(F.col('rating_rating')))
all_ratings_avg_filtered = all_ratings_avg.filter(F.col("size") >= 20)

Below is a table of the top 10 anime when using every rating available from both anime and ratings csvs, animes with less than 20 ratings omitted, in descending order

In [22]:
all_ratings_avg_filtered.sort(col("avg_rating").desc()).show(n=10)

+--------+--------------------+--------------------+--------------------+----------+-----+
|anime_id|              rating|       rating_rating|         all_ratings|avg_rating| size|
+--------+--------------------+--------------------+--------------------+----------+-----+
|   32281|[9.37, 9.37, 9.37...|[9, 5, 8, 10, 10,...|[9.37, 9.37, 9.37...|  9.398156| 1961|
|   28977|[9.25, 9.25, 9.25...|[10, 10, 7, 10, 8...|[9.25, 9.25, 9.25...|  9.349748| 1188|
|    5114|[9.26, 9.26, 9.26...|[10, 10, 9, 10, 1...|[9.26, 9.26, 9.26...|   9.29137|21494|
|     820|[9.11, 9.11, 9.11...|[10, 10, 10, 6, 9...|[9.11, 9.11, 9.11...|  9.249894|  803|
|    9969|[9.16, 9.16, 9.16...|[6, 9, 7, 8, 10, ...|[9.16, 9.16, 9.16...|  9.216276| 3115|
|    9253|[9.17, 9.17, 9.17...|[9, 7, 10, 9, 10,...|[9.17, 9.17, 9.17...|  9.215663|17151|
|   11061|[9.13, 9.13, 9.13...|[9, 9, 8, 10, 9, ...|[9.13, 9.13, 9.13...|  9.182293| 7477|
|   32935|[9.15, 9.15, 9.15...|[10, 9, 10, 10, 9...|[9.15, 9.15, 9.15...|  9.160742| 1038|

now to find top 10 anime using anime csv ratings only

In [23]:
grouped_ratings = outer_join_df_filtered.groupBy('anime_id').agg({'rating': 'collect_list'}).toDF('anime_id', 'rating')

In [24]:
grouped_ratings_anime_rating = grouped_ratings.withColumn("avg_rating", array_mean("rating"))

In [25]:
grouped_ratings_anime_rating = grouped_ratings_anime_rating.withColumn("size", F.size(F.col('rating')))
grouped_ratings_anime_rating_filtered = grouped_ratings_anime_rating.filter(F.col("size") >= 20)

Table below shows the top 10 anime's based on ratings found in the user ratings.csv for animes with more than 20 reviews.

In [28]:
grouped_ratings_anime_rating_filtered.sort(col("rating").desc()).show(n=10)

+--------+--------------------+----------+-----+
|anime_id|              rating|avg_rating| size|
+--------+--------------------+----------+-----+
|   32281|[9.37, 9.37, 9.37...|      9.37| 1961|
|    5114|[9.26, 9.26, 9.26...|      9.26|21494|
|   28977|[9.25, 9.25, 9.25...|      9.25| 1188|
|    9253|[9.17, 9.17, 9.17...|      9.17|17151|
|    9969|[9.16, 9.16, 9.16...|      9.16| 3115|
|   32935|[9.15, 9.15, 9.15...|      9.15| 1038|
|   11061|[9.13, 9.13, 9.13...|      9.13| 7477|
|   15417|[9.11, 9.11, 9.11...|      9.11| 2126|
|     820|[9.11, 9.11, 9.11...|      9.11|  803|
|   15335|[9.1, 9.1, 9.1, 9...|       9.1| 2147|
+--------+--------------------+----------+-----+
only showing top 10 rows



finding top 10 anime using ratings csv only

In [25]:
grouped_ratings_ratings = outer_join_df_filtered.groupBy('anime_id').agg({'rating_rating': 'collect_list'}).toDF('anime_id', 'rating_rating')

In [26]:
grouped_ratings_ratings = grouped_ratings_ratings.withColumn("avg_rating", array_mean("rating_rating"))

In [27]:
grouped_ratings_ratings = grouped_ratings_ratings.withColumn("size", F.size(F.col('rating_rating')))
grouped_ratings_ratings_filtered = grouped_ratings_ratings.filter(F.col("size") >= 20)

Table below shows the top 10 anime's based on ratings found in the user ratings .csv for animes with more than 20 reviews.

In [39]:
grouped_ratings_ratings_filtered.sort(col("avg_rating").desc()).show(n=10)

+--------+--------------------+-----+----------+
|anime_id|       rating_rating| size|avg_rating|
+--------+--------------------+-----+----------+
|   28977|[10, 10, 7, 10, 8...| 1188|  9.449495|
|   32281|[9, 5, 8, 10, 10,...| 1961|  9.426313|
|     820|[10, 10, 10, 6, 9...|  803|  9.389789|
|    5114|[10, 10, 9, 10, 1...|21494| 9.3227415|
|    9969|[6, 9, 7, 8, 10, ...| 3115| 9.2725525|
|    9253|[9, 7, 10, 9, 10,...|17151|  9.261326|
|     918|[9, 7, 9, 10, 10,...| 4264|  9.236398|
|   11061|[9, 9, 8, 10, 9, ...| 7477|  9.234586|
|   15417|[6, 8, 10, 8, 10,...| 2126|  9.202258|
|   15335|[6, 10, 10, 9, 8,...| 2147|   9.19143|
+--------+--------------------+-----+----------+
only showing top 10 rows



as expected some variation exists depending on how we measure although of course we expect the the ratings.csv top 10 to more closely match the combined total as there are ~6m entries in this dataset compared to 12k for anime.csv

so above we have all the ratings for each anime, need to rank them, should first look at how many ratings each anime has and whether it should be included

now to work out most popular genre

In [14]:
genre_df = outer_join_df_filtered.groupBy('genre').agg({'rating_rating': 'collect_list'}).toDF('genre', 'rating_rating')
genre_df = genre_df.na.drop()

In [17]:
grouped_ratings_ratings = grouped_ratings_ratings.withColumn("size", F.size(F.col('rating_rating')))
grouped_ratings_ratings_filtered = grouped_ratings_ratings.filter(F.col("size") >= 20)

NameError: ignored

In [15]:
# group genres using all ratings available, all_ratings will be applied to each genre after the genre column is exploded
col_list = ["rating", "rating_rating"]
genres = outer_join_df_filtered.groupBy("genre").agg( *[F.collect_list(column).alias(column) for column in col_list if column !='genre' ]) 
genres = genres.withColumn("all_ratings", concat(col("rating"), col("rating_rating")))
genres_avg = genres.withColumn("avg_rating", array_mean("all_ratings"))

In [31]:
genres_avg = genres_avg.withColumn("size", F.size(F.col('all_ratings')))
genres_avg_filtered = genres_avg.filter(F.col("size") >= 20)

In [32]:
genres_avg_filtered.show()

+--------------------+--------------------+--------------------+--------------------+----------+-----+
|               genre|              rating|       rating_rating|         all_ratings|avg_rating| size|
+--------------------+--------------------+--------------------+--------------------+----------+-----+
|                null|[5.52, 5.52, 5.52...|[6, 5, 6, 6, 6, 5...|[5.52, 5.52, 5.52...|  4.990052|  191|
|   Action, Adventure|[5.04, 5.25, 5.04...|[5, 5, 5, 6, 4, 7...|[5.04, 5.25, 5.04...|  7.245631|12116|
|Action, Adventure...|[6.86, 6.86, 6.86...|[9, 8, 9, 5, 7, 6...|[6.86, 6.86, 6.86...| 6.8742723|  646|
|Action, Adventure...|[6.52, 6.97, 6.52...|[7, 5, 8, 7, 6, 7...|[6.52, 6.97, 6.52...| 6.9250426| 1406|
|Action, Adventure...|[7.13, 7.13, 7.13...|[9, 6, 7, 8, 7, 1...|[7.13, 7.13, 7.13...|  7.203412|  932|
|Action, Adventure...|[7.89, 7.89, 7.89...|[1, 4, 8, 8, 9, 1...|[7.89, 7.89, 7.89...|  8.098305|21252|
|Action, Adventure...|[7.66, 7.66, 7.66...|[8, 8, 8, 6, 5, 8...|[7.66, 7.

In [41]:
df_genre.printSchema()

root
 |-- genre: string (nullable = true)
 |-- avg_rating: float (nullable = true)



In [16]:
genres_avg = genres_avg.dropna()

In [17]:
genres_avg = genres_avg.drop("rating","rating_rating","size", "all_ratings")

In [41]:
genres_avg.printSchema()

root
 |-- genre: string (nullable = true)
 |-- avg_rating: float (nullable = true)



In [34]:
genres_avg.count()

3229

In [18]:
# new df with genres and avg ratings
df_genre_expl = genres_avg.withColumn("genre", explode(split("genre", "[,]")))

In [36]:
df_genre_expl.count()

13889

In [38]:
genre_rdd = genres_avg.rdd

In [ ]:
genre_map = genre_rdd.flatMap(lambda x: )

In [ ]:
map1=genres_avg.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()

In [43]:
df_genre_expl.show()

+----------+----------+
|     genre|avg_rating|
+----------+----------+
|    Action|  7.245631|
| Adventure|  7.245631|
|    Action| 6.8742723|
| Adventure| 6.8742723|
|      Cars| 6.8742723|
|    Sci-Fi| 6.8742723|
|    Action| 6.9250426|
| Adventure| 6.9250426|
|    Comedy| 6.9250426|
|    Action|  7.203412|
| Adventure|  7.203412|
|    Comedy|  7.203412|
|    Demons|  7.203412|
|     Drama|  7.203412|
|     Ecchi|  7.203412|
|    Horror|  7.203412|
|   Mystery|  7.203412|
|   Romance|  7.203412|
|    Sci-Fi|  7.203412|
|    Action|  8.098305|
+----------+----------+
only showing top 20 rows



In [19]:
# groupby genres then
df_genre_expl_gouped = df_genre_expl.groupBy('genre').agg({'avg_rating': 'collect_list'}).toDF('genre', 'avg_rating')

In [46]:
df_genre_expl_gouped.show()

+--------------+--------------------+
|         genre|          avg_rating|
+--------------+--------------------+
|        Seinen|[5.99, 6.4509945,...|
|        Sports|[7.0794444, 6.321...|
|         Harem|[6.7053237, 7.489...|
|    Shounen Ai|[7.330708, 6.8312...|
|         Ecchi|[7.203412, 6.1662...|
|      Military|[8.290691, 7.3120...|
|      Dementia|[4.278857, 5.8758...|
|       Romance|[6.99633, 7.11504...|
|    Historical|[6.3814287, 6.792...|
|         Magic|[6.3920383, 5.85,...|
|      Thriller|          [6.581241]|
| Psychological|[6.965, 6.9952435...|
|         Josei|    [8.52355, 5.835]|
|       Romance|[7.203412, 8.0983...|
|        Demons|[7.203412, 8.0983...|
|          Yuri|[4.325, 5.7308044...|
|     Adventure|[6.74496, 4.3, 5....|
|   Super Power|         [4.8133335]|
|  Martial Arts|[7.7173076, 7.121...|
|        Sports|          [7.049756]|
+--------------+--------------------+
only showing top 20 rows



In [24]:
genre_ratings_avg = df_genre_expl_gouped.withColumn("total_avg_rating", array_mean("avg_rating"))

In [26]:
genre_ratings_avg_nanless = genre_ratings_avg.dropna

In [25]:
genre_ratings_avg.sort(col("total_avg_rating").desc()).show(n=10)

+--------------+--------------------+----------------+
|         genre|          avg_rating|total_avg_rating|
+--------------+--------------------+----------------+
|         Josei|[7.11212, 7.89129...|       7.5873466|
|       Mystery|[5.840294, 8.7018...|       7.4956846|
|       Shounen|[7.514094, 7.1785...|        7.479605|
|      Thriller|[7.6649137, 8.131...|       7.3842406|
|        Sci-Fi|[6.772781, 7.7305...|       7.3129444|
|         Drama|[7.203412, 7.6831...|        7.256711|
|        School|[7.062824, 7.2371...|        7.248503|
|       Mystery|[7.203412, 8.2928...|        7.221534|
| Psychological|[6.965, 6.9952435...|        7.221369|
| Slice of Life|[7.3, 7.761316, 6...|       7.2208643|
+--------------+--------------------+----------------+
only showing top 10 rows



**4. Student Activity: Design the recommendation system. Remember to split the dataset into training and testing to validate your recommendation model. This section would help you in answering question 5**

Q5. Design a collaborative filter-based recommendation system. 
# https://rpubs.com/bsvmelo/639969

In [55]:
# ratings for colloborative filtering and genre + type for content based recommendation 

**Student Activity: Analyse the output of the test dataset here.**

Q6. Give example of best three anime recommendations for minimum of 10 users.

In [ ]:
spark.stop()